### Проверка того, что наше api работает

Перед началом необходимо собрать docker-образ и запустить контейнер

1. Следовать инструкциям https://github.com/fimochka-sudo/GB_docker_flask_example/blob/main/README.md
2. Далее уже запускать код ниже

In [1]:
import pandas as pd
from sklearn.metrics import roc_auc_score,roc_curve,scorer
from urllib import request, parse

X_test = pd.read_csv("X_test.csv")
y_test = pd.read_csv("y_test.csv")

/home/sil/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)


In [2]:
# X_test.columns
columns = ['Annual Income', 'Number of Open Accounts', 'Years of Credit History',
       'Maximum Open Credit', 'Current Loan Amount', 'Current Credit Balance',
       'Monthly Debt',
       'CreditScore_small', 'CreditScore_large',
       'Home_Ownership_int', 'Years_in_current_job_int', 'Purpose_int',
       'Term_int', 'Tax_Liens_int', 'Number_of_Credit_Problems_int']

In [3]:
# X_test[columns].head(3)

In [4]:
import urllib.request
import json      

def get_prediction(Annual_Income,
                   
                   Number_of_Open_Accounts,
                   Years_of_Credit_History,
                   Maximum_Open_Credit,
                   Current_Loan_Amount,
                   Current_Credit_Balance,
                   Monthly_Debt,
                   CreditScore_small,
                   CreditScore_large,
                   
                   Home_Ownership_int,
                   Years_in_current_job_int,
                   Purpose_int,
                   Term_int,
                   Tax_Liens_int,
                   Number_of_Credit_Problems_int):
    

    body = {'Annual Income': str(Annual_Income),

            'Number of Open Accounts': str(float(Number_of_Open_Accounts)),
            'Years of Credit History': str(float(Years_of_Credit_History)), 
            'Maximum Open Credit': str(float(Maximum_Open_Credit)), 
            'Current Loan Amount': str(float(Current_Loan_Amount)), 
            'Current Credit Balance': str(float(Current_Credit_Balance)),

            'Monthly Debt': str(float(Monthly_Debt)),
            'CreditScore_small': str(float(CreditScore_small)),
            'CreditScore_large': str(float(CreditScore_large)),
            
            'Home_Ownership_int': str(int(Home_Ownership_int)),
            'Years_in_current_job_int': str(int(Years_in_current_job_int)), 
            'Purpose_int': str(int(Purpose_int)), 
            'Term_int': str(int(Term_int)), 
            'Tax_Liens_int': str(int(Tax_Liens_int)),
            'Number_of_Credit_Problems_int': str(int(Number_of_Credit_Problems_int))
            } 
    
#     for x in body:
#         print (x,':',body[x])
    
    jsondata = json.dumps(body)
    jsondataasbytes = jsondata.encode('utf-8')   # needs to be bytes
    # print (jsondataasbytes)
    # myurl = "http://0.0.0.0:8180/predict"
    myurl = "http://0.0.0.0:8182/predict"
    req = urllib.request.Request(myurl)
    req.add_header('Content-Type', 'application/json; charset=utf-8')
    req.add_header('Content-Length', len(jsondataasbytes))

    response = urllib.request.urlopen(req, jsondataasbytes)

    return json.loads(response.read())['predictions']

In [5]:
#  get_prediction(0.616627, 3, 8, 14, 1, 0, 0)

In [6]:
X_test[columns].head(1)

,Annual Income,Number of Open Accounts,Years of Credit History,Maximum Open Credit,Current Loan Amount,Current Credit Balance,Monthly Debt,CreditScore_small,CreditScore_large,Home_Ownership_int,Years_in_current_job_int,Purpose_int,Term_int,Tax_Liens_int,Number_of_Credit_Problems_int
0,0.616627,0.380774,-0.345022,0.41448,-0.506407,0.775597,0.342168,0.381853,-0.237124,3,8,14,1,0,0


In [7]:
get_prediction(0.616627,
               0.380774,
               -0.345022,
               0.414480,
               -0.506407,
               0.775597,
               0.342168,
               0.381853,
               -0.237124,
               3,
               8,
               14,
               1,
               0,
               0)

0.13721633456135737

In [8]:
%%time
predictions = X_test.iloc[:100].apply(lambda x: get_prediction(x[0], 
                                                             x[1],
                                                             x[2],
                                                             x[3],
                                                             x[4],
                                                             x[5],
                                                             x[6],
                                                             x[7],
                                                             x[8],
                                                             x[9],
                                                             x[10],
                                                             x[11],
                                                             x[12],
                                                             x[13],
                                                             x[14]), 1)

CPU times: user 213 ms, sys: 65.3 ms, total: 279 ms
Wall time: 5.55 s


In [9]:
roc_auc_score(y_score=predictions.values, y_true=y_test.iloc[:100])

0.7484215638659544

Давайте подумаем, где наше api может сломаться

In [10]:
# get_prediction("", "", None)